# Webscraping 

Web scraping the Wine Spectator website "winesectator.com" for oregon wine reviews and wine information.

Started with my Wine Enthusiest code.



In [1]:
import re
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep



In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}

In [8]:
browser = Browser('chrome', **executable_path, headless=False)

In [9]:
url = "https://www.winespectator.com/wine/searchv1?submitted=Y&page=1&winery=&text_search_flag=everything&search_by=all&scorelow=-1&scorehigh=-1&pricelow=-1&pricehigh=-1&case_prod=null_case_prod&accolade%5B%5D=&taste_date=&issue_date=&issue_year=&varietal%5B%5D=allredwines&regions%5B%5D=v18%3AOregon&vintage%5B%5D=&size=50&sort_by=score&sort_dir=desc"

In [10]:
browser.visit(url)  ## make sure the browser is full screen. it will matter for seach results

In [12]:
soup = bs(browser.html, 'html.parser')
soup

<html class="js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths" lang="en"><!--<![endif]--><head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<script async="" src="https://script.crazyegg.com/pages/versioned/common-scripts/dcc9a1fbfa71e970c45301eb49f6db43.js" type="text/javascript"></script><script async="" src="https://connect.facebook.net/signals/config/1606454569699660?v=2.9.89&amp;r=stable"></script><script async="" src="https://connect.facebook.net/en_

In [13]:
## Too many results for viewing, 8,000+ with a limit of 3,000 to view. will have to scrape from several searches.
## Narrow the search by grape
adv = soup.find("div", id="adv-controls")
div = adv.find_all("div", class_="col-md-4 form-group")[0]
grapes = dict()
varietals = div.find_all("option")

for option in varietals:
    if "null" not in option.get("value"):
        if ("allredwines" not in option.get("value")) and  ("allwhitewines" not in option.get("value")):
            grapes[option.text] = option.get("value")

print(grapes)

{'Barbera': 'v11:Barbera', 'Cabernet Sauvignon & Blends': 'Cabernet_Sauvignon', 'Cabernet Franc': 'v11:Cabernet_Franc', 'Gamay': 'v11:Gamay', 'Grenache': 'v11:Grenache', 'Malbec': 'v11:Malbec', 'Merlot': 'Merlot', 'Mourvèdre': 'v11:Mourvèdre', 'Petite Sirah': 'v11:Petite_Sirah', 'Pinot Noir': 'v11:Pinot_Noir', 'Pinotage': 'v11:Pinotage', 'Sangiovese': 'v11:Sangiovese', 'Shiraz/Syrah': 'Shiraz', 'Tempranillo': 'v11:Tempranillo', 'Zinfandel': 'Zinfandel', 'Other Red Wines': 'otherredwines', 'Chardonnay': 'v11:Chardonnay', 'Chenin Blanc': 'v11:Chenin_Blanc', 'Gewürztraminer': 'v11:Gewürztraminer', 'Grüner Veltliner': 'v11:Grüner_Veltliner', 'Marsanne': 'v11:Marsanne', 'Pinot Blanc': 'v11:Pinot_Blanc', 'Pinot Gris': 'v11:Pinot_Gris', 'Riesling': 'v11:Riesling', 'Sauvignon Blanc': 'Sauvignon_Blanc', 'Sémillon': 'Semillon', 'Viognier': 'v11:Viognier', 'Other White Wines': 'otherwhitewines', 'Rosé Wines': 'allblushwines', 'Sparkling Wines': 'allsparklingwines', 'Dessert Wines': 'alldessertwin

In [ ]:
## What will be the results page format?
cellar = []
grape_blurb = list()
results = soup.find_all("tr", attrs={"data-cyd":"wineResultContainer"})
for row in results[:10]: #[checkbox, chevron, name/link, vintage, score, release price]
    td = row.find_all("td")
    vintage = td[-3]
    score = td[-2]
    release_price = td[-1]
    link = "https://www.winespectator.com"+td[-4].find("a").get("href")
#     wine spectator's unique wine id
    txt = re.search("id/(\d+)/", link)
    wine_id = txt.group()[3:-1]
    
    ## visit the specific review link
    browser.visit(link)
    sleep(10)
    review = bs(browser.html, "html.parser")
    wine = review.find('div', class_="ws-dd-grid-content__text").find("h1").find_all("span")[1].text.split('\n')[0]
    winery = review.find('div', class_="ws-dd-grid-content__text").find("h1").find_all("span")[0].text
    review_text = review.find('div', class_="ws-dd-grid-content__text").find("p").text
    review_location = review.find('div', class_="ws2-dd-details-grid").find("a").find("div").find_all("p")[0].text.replace("\n","")
    review_location += " "+ review.find('div', class_="ws2-dd-details-grid").find("a").find("div").find_all("p")[1].text
    grape = review.find('div', class_="ws2-dd-grape-grid").find("h3").text
    grape_text_short=review.find('div', class_="ws2-dd-grape-grid").find("p").text.replace("\n", "").replace("\t", "")
    grape_link = "https://www.winespectator.com"+review.find('div', class_="ws2-dd-grape-grid").find("a").get("href")
    grape_blurb.append({"grape_name":grape, "info_link":grape_link, "short_text":grape_text_short})
    region = review.find('div', class_="ws2-dd-summary-grid__region").find("p").text.replace("\n", "").split(" / ")
    cellar.append({"vintage":vintage, "score":score, "release_price":release_price, "bottle_name":wine, "winery":winery,
                   "review_text":review_text, "Magazine":review_location, "review_link":link,
                   "grape":grape, "region":region, "wine_id":wine_id})
cellar

In [10]:
pages = int(soup.find_all("div", class_="article__byline")[0].find("strong").get("data-results"))/50

if pages != int(pages):
    last_page = int(pages) +1
else:
    last_page=int(pages)

    

In [33]:
varietals = list(grapes.keys())
varietals.sort()
varietals.remove('Pinot Noir')
varietals.append('Pinot Noir')
varietals

['Barbera',
 'Cabernet Franc',
 'Cabernet Sauvignon & Blends',
 'Chardonnay',
 'Chenin Blanc',
 'Dessert Wines',
 'Gamay',
 'Gewürztraminer',
 'Grenache',
 'Grüner Veltliner',
 'Malbec',
 'Marsanne',
 'Merlot',
 'Mourvèdre',
 'Other Red Wines',
 'Other White Wines',
 'Petite Sirah',
 'Pinot Blanc',
 'Pinot Gris',
 'Pinotage',
 'Riesling',
 'Rosé Wines',
 'Sangiovese',
 'Sauvignon Blanc',
 'Shiraz/Syrah',
 'Sparkling Wines',
 'Sémillon',
 'Tempranillo',
 'Viognier',
 'Zinfandel',
 'Pinot Noir']

In [14]:
# Global Variables
html_grapes = set()
grapes = set()
cellar = []
grape_blurb = set()
logout = False

## grab the Varietals
url = f"https://www.winespectator.com/wine/searchv1?submitted=Y&page=1&winery=&text_search_flag=everything&search_by=all&scorelow=-1&scorehigh=-1&pricelow=-1&pricehigh=-1&case_prod=null_case_prod&accolade%5B%5D=&taste_date=&issue_date=&issue_year=&varietal%5B%5D=allredwines&regions%5B%5D=v18%3AOregon&vintage%5B%5D=&size=50&sort_by=score&sort_dir=desc"
browser.visit(url)
soup = bs(browser.html, 'html.parser')
adv = soup.find("div", id="adv-controls")
div = adv.find_all("div", class_="col-md-4 form-group")[0]
grapes = dict()
varietals = div.find_all("option")
saved_links = set()


for option in varietals:
    if "null" not in option.get("value"):
        if ("allredwines" not in option.get("value")) and  ("allwhitewines" not in option.get("value")):
            grapes[option.text] = option.get("value")

# make sure pinot noir is the last varietal grabbed of every year because it is the largest result set
varietals = list(grapes.keys())
varietals.sort()
varietals.remove('Pinot Noir')
varietals.append('Pinot Noir')
varietals

## vintage years to save
vintages = [2011, 2010,2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2020]

for n in vintages:
## iterate through the search results for each grape within that vintage year
    for key in varietals:
        
        url = f"https://www.winespectator.com/wine/searchv1?submitted=Y&page=1&winery=&text_search_flag=everything&search_by=all&scorelow=-1&scorehigh=-1&pricelow=-1&pricehigh=-1&case_prod=null_case_prod&accolade%5B%5D=&taste_date=&issue_date=&issue_year=&varietal%5B%5D={grapes[key]}&regions%5B%5D=v18%3AOregon&vintage%5B%5D={n}&size=15&sort_by=score&sort_dir=desc"
        browser.visit(url)
        sleep(5)
        soup = bs(browser.html, "html.parser")
        try:
            pages = int(soup.find_all("div", class_="article__byline")[0].find("strong").get("data-results"))/15
        except: 
            sleep(3)
            continue    
         
        if pages != int(pages):
            last_page = int(pages) +1
        else:
            last_page=int(pages)
        
        print(f"{n} {key} has {last_page} pages")
        for i in range(1,(last_page+1)):
            if i > 1:
                url = f"https://www.winespectator.com/wine/searchv1?submitted=Y&page={i}&winery=&text_search_flag=everything&search_by=all&scorelow=-1&scorehigh=-1&pricelow=-1&pricehigh=-1&case_prod=null_case_prod&accolade%5B%5D=&taste_date=&issue_date=&issue_year=&varietal%5B%5D={grapes[key]}&regions%5B%5D=v18%3AOregon&vintage%5B%5D={n}&size=15&sort_by=score&sort_dir=desc"
                browser.visit(url)
                sleep(5)
            else:
                pass

            results = soup.find_all("tr", attrs={"data-cyd":"wineResultContainer"})
            for row in results: #[checkbox, chevron, name/link, vintage, score, release price]
                td = row.find_all("td")
                if len(td) != 0: 
                    vintage = td[-3].text
                    score = td[-2].text
                    release_price = td[-1].text
                    link = "https://www.winespectator.com"+td[-4].find("a").get("href")
                    if link in saved_links:
                        continue
                    else:
                        saved_links.add(link)
                    
                    
#                     wine spectators unique wine id
                    txt = re.search("id/(\d+)/", link)
                    wine_id = txt.group()[3:-1]
                    
                    ## visit the specific review link
                    browser.visit(link)
                    sleep(6)
                    review = bs(browser.html, "html.parser")
                    wine = review.find('div', class_="ws-dd-grid-content__text").find("h1").find_all("span")[1].text.split('\n')[0]
                    winery = review.find('div', class_="ws-dd-grid-content__text").find("h1").find_all("span")[0].text
                    review_text = review.find('div', class_="ws-dd-grid-content__text").find("p").text
                    review_location = review.find('div', class_="ws2-dd-details-grid").find("a").find("div").find_all("p")[0].text.replace("\n","")
                    review_location += " "+ review.find('div', class_="ws2-dd-details-grid").find("a").find("div").find_all("p")[1].text
                    
                    region = review.find('div', class_="ws2-dd-summary-grid__region").find("p").text.replace("\n", "").split(" / ")
                    cellar.append({"vintage":vintage, "score":score, "release_price":release_price, "bottle_name":wine, "winery":winery,
                                   "review_text":review_text, "Magazine":review_location, "review_link":link,
                                    "region":region, "source":"Wine Spectator", "varietal":key, "wine_id":wine_id})
                    try:
                        grape_link = "https://www.winespectator.com"+review.find('div', class_="ws2-dd-grape-grid").find("a").get("href")
                        grape_blurb.add(grape_link)
                    except: 
                        pass
                else:
                     ## the website logged me out and now have to log back in
                    logout = True
                    break

                    
                    
        if logout==True:
            break
            
            
            
    if logout==True:
            break            

      

2011 Barbera has 0 pages
2011 Cabernet Franc has 1 pages
2011 Cabernet Sauvignon & Blends has 1 pages
2011 Chardonnay has 4 pages
2011 Chenin Blanc has 0 pages
2011 Dessert Wines has 1 pages
2011 Gamay has 0 pages
2011 Gewürztraminer has 0 pages
2011 Grenache has 1 pages
2011 Malbec has 1 pages
2011 Marsanne has 0 pages
2011 Mourvèdre has 0 pages
2011 Other Red Wines has 1 pages
2011 Other White Wines has 2 pages
2011 Petite Sirah has 0 pages
2011 Pinot Blanc has 1 pages
2011 Pinot Gris has 4 pages
2011 Pinotage has 0 pages
2011 Riesling has 2 pages
2011 Rosé Wines has 1 pages
2011 Sangiovese has 0 pages
2011 Sauvignon Blanc has 1 pages
2011 Shiraz/Syrah has 1 pages
2011 Sparkling Wines has 1 pages
2011 Sémillon has 0 pages
2011 Tempranillo has 1 pages
2011 Viognier has 1 pages
2011 Zinfandel has 0 pages
2011 Pinot Noir has 24 pages
2010 Barbera has 0 pages
2010 Cabernet Franc has 1 pages
2010 Cabernet Sauvignon & Blends has 1 pages
2010 Chardonnay has 3 pages
2010 Chenin Blanc has 0 p

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=107.0.5304.106)


In [19]:
len(cellar)

252

In [26]:
spam = pd.DataFrame(cellar)
spam

,vintage,score,release_price,bottle_name,winery,review_text,Magazine,review_link,region,source,varietal,wine_id
0,2011,91,$40,Cabernet Franc Rogue Valley,Griffin Creek,"Soft, ripe, generous and precise, this open-te...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Southern Oregon, Rogue Valley]",Wine Spectator,Cabernet Franc,379875
1,2011,86,$40,Arsenal Rogue Valley,Quady North,"A snappy red, if modest in scale, with blackbe...","Magazine Issue Feb 02, 2017",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Southern Oregon, Rogue Valley]",Wine Spectator,Cabernet Franc,429948
2,2011,90,$28,Cabernet Sauvignon Rogue Valley,Del Rio,"A vibrant, medium-weight red, with ripe plum a...","Magazine Issue Feb 28, 2014",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Southern Oregon, Rogue Valley]",Wine Spectator,Cabernet Sauvignon & Blends,357733
3,2011,86,$40,Cabernet Sauvignon Rogue Valley,Griffin Creek,"Lean, spicy and firmly tannic, offering billow...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Southern Oregon, Rogue Valley]",Wine Spectator,Cabernet Sauvignon & Blends,379943
4,2011,94,$125,Chardonnay Dundee Hills Récolte Grand Cru,Domaine Serene,"Tangy, with vibrant acidity and expressive, ri...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Willamette Valley, Dundee Hills]",Wine Spectator,Chardonnay,380170
...,...,...,...,...,...,...,...,...,...,...,...,...
247,2010,86,$28,Tempranillo Rogue Valley,Ledger David,Scratchy tannins underlie a generous core of r...,"Magazine Issue Feb 28, 2014",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Southern Oregon, Rogue Valley]",Wine Spectator,Tempranillo,357731
248,2010,85,$38,Tempranillo Rogue Valley,Griffin Creek,"Supple and expressive, reining in currant and ...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Southern Oregon, Rogue Valley]",Wine Spectator,Tempranillo,379947
249,2010,87,$24,Viognier Yamhill-Carlton District Deux Vert Vi...,Et Fille,"Spicy and appealing in flavor, offering pear a...","Magazine Issue Aug 31, 2012",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Willamette Valley, Yamhill-Carlton Di...",Wine Spectator,Viognier,323823
250,2010,95,$45,Pinot Noir Dundee Hills ??? Vineyards,Ayoub,"Vibrant, elegant and complex, with pinpoint fo...","Magazine Issue Dec 15, 2012",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Willamette Valley, Dundee Hills]",Wine Spectator,Pinot Noir,331824


## Data Update:
- The first scrape was aimed to gain 4k+ reviews of the 8k+ the website claims to have for oregon wine. The first attempt resulted in grabing 4K+ reviews but most of them were duplicates, only 373 were unique. Looking over the code shows that the automated browser size matters for gathering the grape varietals. (old_wine_spectator_reviews.csv)
- The second attempt was making sure the window was maximized before scraping, did this manually when the window opened. After calculating the estimated time per review count scraped I stopped the script when i was expecting to have just under 2k unique reviews. There were 1900 reviews, 689 were unique. (old1_wine_spectator_reviews.csv)
- The third attempt included a check for duplicates, were the previous 2 did not, by checking to see if the unique review link was in a set() of proviously scrapped reviews. This attempt resulted in 252 unique reviews. 

#### Combined the 3 unique data sets from the different attempts to scrape and check for a total count of unique reviews.


In [35]:
spam = pd.concat([spam, pd.read_csv("old_wine_spectator_reviews.csv").drop_duplicates(), pd.read_csv("old1_spectator_reviews.csv").drop_duplicates()], axis=0, ignore_index=False)

In [46]:
egg = set()
number = []
for i in spam.index:
    test = str()
    for col in spam.columns:
        test += str(spam[col][i])
    
    if test not in egg:
        egg.add(test)
        number.append(i)

len(egg)    
len(number)

986

In [51]:
number.sort()
len(number)

986

In [52]:
spam.loc[number, ]

,vintage,score,release_price,bottle_name,winery,review_text,Magazine,review_link,region,source,varietal,wine_id,Region
0,2011,91,$40,Cabernet Franc Rogue Valley,Griffin Creek,"Soft, ripe, generous and precise, this open-te...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Southern Oregon, Rogue Valley]",Wine Spectator,Cabernet Franc,379875,NaN
0,2019,96,$65,Pinot Noir Yamhill-Carlton Résonance Vineyard,Résonance,"Impressive and compelling, this Pinot epitomiz...","Magazine Issue Jul 31, 2022",https://www.winespectator.com/wine/wine-detail...,NaN,Wine Spectator,NaN,NaN,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt..."
0,2021,90,$20,Chardonnay Willamette Valley Inox Unoaked,Chehalem,"A dynamic white, with spirited lemon and lime ...","Magazine Issue Nov 15, 2022",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator,Chardonnay,1288804,NaN
1,2011,86,$40,Arsenal Rogue Valley,Quady North,"A snappy red, if modest in scale, with blackbe...","Magazine Issue Feb 02, 2017",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Southern Oregon, Rogue Valley]",Wine Spectator,Cabernet Franc,429948,NaN
1,2019,95,$80,Pinot Noir Chehalem Mountains Silice,Bergström,"Polished, rich and luscious, offering vibrant ...",Magazine Issue Web Only - 2021,https://www.winespectator.com/wine/wine-detail...,NaN,Wine Spectator,NaN,NaN,"['Oregon', 'Willamette Valley', 'Chehalem Moun..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4468,2020,93,$40,Pinot Noir Ribbon Ridge,Norris,"Supple and detailed, with multilayered red plu...","Magazine Issue Nov 15, 2022",https://www.winespectator.com/wine/wine-detail...,NaN,Wine Spectator,NaN,NaN,"['Oregon', 'Willamette Valley', 'Ribbon Ridge']"
4469,2020,93,$40,Pinot Noir Dundee Hills Estate,Stoller,This lovely red is elegantly complex and polis...,"Magazine Issue Oct 15, 2022",https://www.winespectator.com/wine/wine-detail...,NaN,Wine Spectator,NaN,NaN,"['Oregon', 'Willamette Valley', 'Dundee Hills']"
4470,2020,92,$50,Chardonnay Willamette Valley Affinités,Nicolas-Jay,"Handsomely built, with deep and multilayered f...","Magazine Issue Nov 15, 2022",https://www.winespectator.com/wine/wine-detail...,NaN,Wine Spectator,NaN,NaN,"['Oregon', 'Willamette Valley']"
4471,2020,92,$58,Pinot Noir Yamhill-Carlton Domaine Danielle La...,Soléna,"A structured red, with a lively backbone of ac...","Magazine Issue Jul 31, 2022",https://www.winespectator.com/wine/wine-detail...,NaN,Wine Spectator,NaN,NaN,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt..."


In [53]:
spam[]

,vintage,score,release_price,bottle_name,winery,review_text,Magazine,review_link,region,source,varietal,wine_id,Region
0,2011,91,$40,Cabernet Franc Rogue Valley,Griffin Creek,"Soft, ripe, generous and precise, this open-te...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Southern Oregon, Rogue Valley]",Wine Spectator,Cabernet Franc,379875,NaN
1,2011,86,$40,Arsenal Rogue Valley,Quady North,"A snappy red, if modest in scale, with blackbe...","Magazine Issue Feb 02, 2017",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Southern Oregon, Rogue Valley]",Wine Spectator,Cabernet Franc,429948,NaN
2,2011,90,$28,Cabernet Sauvignon Rogue Valley,Del Rio,"A vibrant, medium-weight red, with ripe plum a...","Magazine Issue Feb 28, 2014",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Southern Oregon, Rogue Valley]",Wine Spectator,Cabernet Sauvignon & Blends,357733,NaN
3,2011,86,$40,Cabernet Sauvignon Rogue Valley,Griffin Creek,"Lean, spicy and firmly tannic, offering billow...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Southern Oregon, Rogue Valley]",Wine Spectator,Cabernet Sauvignon & Blends,379943,NaN
4,2011,94,$125,Chardonnay Dundee Hills Récolte Grand Cru,Domaine Serene,"Tangy, with vibrant acidity and expressive, ri...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"[Oregon, Willamette Valley, Dundee Hills]",Wine Spectator,Chardonnay,380170,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1708,2015,90,$45,Chardonnay Dundee Hills,Knudsen,"Crisp and harmonious, with lemon blossom and a...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,434673,NaN
1709,2015,90,$40,Chardonnay Willamette Valley Reserve,Ponzi,"Graceful and polished, with expressive pear, s...","Magazine Issue Jul 31, 2018",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator,Chardonnay,454785,NaN
1710,2015,90,$45,Chardonnay Yamhill-Carlton District Gran Morai...,Raptor Ridge,"Light and polished, showing pretty pear, pinea...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,427274,NaN
1711,2015,90,$60,Chardonnay Yamhill-Carlton District Marsh Esta...,Roco,"Floral and harmonious, with delicate tropical ...",Magazine Issue Web Only - 2018,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,447448,NaN


#### when combined into one dataframe, the drop_duplicates function errors out because there is a list in the dataframe. Save it to a csv and re-read it in.

In [54]:
spam.to_csv("combined_spectator_reviews.csv", index=False)

In [77]:
df = pd.read_csv("combined_spectator_reviews.csv")
df.drop_duplicates()

,vintage,score,release_price,bottle_name,winery,review_text,Magazine,review_link,region,source,varietal,wine_id,Region
0,2011,91,$40,Cabernet Franc Rogue Valley,Griffin Creek,"Soft, ripe, generous and precise, this open-te...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Franc,379875.0,NaN
1,2011,86,$40,Arsenal Rogue Valley,Quady North,"A snappy red, if modest in scale, with blackbe...","Magazine Issue Feb 02, 2017",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Franc,429948.0,NaN
2,2011,90,$28,Cabernet Sauvignon Rogue Valley,Del Rio,"A vibrant, medium-weight red, with ripe plum a...","Magazine Issue Feb 28, 2014",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Sauvignon & Blends,357733.0,NaN
3,2011,86,$40,Cabernet Sauvignon Rogue Valley,Griffin Creek,"Lean, spicy and firmly tannic, offering billow...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Sauvignon & Blends,379943.0,NaN
4,2011,94,$125,Chardonnay Dundee Hills Récolte Grand Cru,Domaine Serene,"Tangy, with vibrant acidity and expressive, ri...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,380170.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,2015,90,$45,Chardonnay Dundee Hills,Knudsen,"Crisp and harmonious, with lemon blossom and a...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,434673.0,NaN
1234,2015,90,$40,Chardonnay Willamette Valley Reserve,Ponzi,"Graceful and polished, with expressive pear, s...","Magazine Issue Jul 31, 2018",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator,Chardonnay,454785.0,NaN
1235,2015,90,$45,Chardonnay Yamhill-Carlton District Gran Morai...,Raptor Ridge,"Light and polished, showing pretty pear, pinea...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,427274.0,NaN
1236,2015,90,$60,Chardonnay Yamhill-Carlton District Marsh Esta...,Roco,"Floral and harmonious, with delicate tropical ...",Magazine Issue Web Only - 2018,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,447448.0,NaN


In [78]:
df["wine_id"] = spam2["review_link"].map(lambda x: int(re.search('id/(\d+)/', x).group()[3:-1]))
df

,vintage,score,release_price,bottle_name,winery,review_text,Magazine,review_link,region,source,varietal,wine_id,Region
0,2011,91,$40,Cabernet Franc Rogue Valley,Griffin Creek,"Soft, ripe, generous and precise, this open-te...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Franc,379875,NaN
1,2011,86,$40,Arsenal Rogue Valley,Quady North,"A snappy red, if modest in scale, with blackbe...","Magazine Issue Feb 02, 2017",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Franc,429948,NaN
2,2011,90,$28,Cabernet Sauvignon Rogue Valley,Del Rio,"A vibrant, medium-weight red, with ripe plum a...","Magazine Issue Feb 28, 2014",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Sauvignon & Blends,357733,NaN
3,2011,86,$40,Cabernet Sauvignon Rogue Valley,Griffin Creek,"Lean, spicy and firmly tannic, offering billow...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Sauvignon & Blends,379943,NaN
4,2011,94,$125,Chardonnay Dundee Hills Récolte Grand Cru,Domaine Serene,"Tangy, with vibrant acidity and expressive, ri...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,380170,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,2015,90,$45,Chardonnay Dundee Hills,Knudsen,"Crisp and harmonious, with lemon blossom and a...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,434673,NaN
1234,2015,90,$40,Chardonnay Willamette Valley Reserve,Ponzi,"Graceful and polished, with expressive pear, s...","Magazine Issue Jul 31, 2018",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator,Chardonnay,454785,NaN
1235,2015,90,$45,Chardonnay Yamhill-Carlton District Gran Morai...,Raptor Ridge,"Light and polished, showing pretty pear, pinea...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,427274,NaN
1236,2015,90,$60,Chardonnay Yamhill-Carlton District Marsh Esta...,Roco,"Floral and harmonious, with delicate tropical ...",Magazine Issue Web Only - 2018,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,447448,NaN


In [68]:
spam2.drop_duplicates()

,vintage,score,release_price,bottle_name,winery,review_text,Magazine,review_link,region,source,varietal,Region,wine_id
0,2011,91,$40,Cabernet Franc Rogue Valley,Griffin Creek,"Soft, ripe, generous and precise, this open-te...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Franc,NaN,379875
1,2011,86,$40,Arsenal Rogue Valley,Quady North,"A snappy red, if modest in scale, with blackbe...","Magazine Issue Feb 02, 2017",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Franc,NaN,429948
2,2011,90,$28,Cabernet Sauvignon Rogue Valley,Del Rio,"A vibrant, medium-weight red, with ripe plum a...","Magazine Issue Feb 28, 2014",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Sauvignon & Blends,NaN,357733
3,2011,86,$40,Cabernet Sauvignon Rogue Valley,Griffin Creek,"Lean, spicy and firmly tannic, offering billow...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Sauvignon & Blends,NaN,379943
4,2011,94,$125,Chardonnay Dundee Hills Récolte Grand Cru,Domaine Serene,"Tangy, with vibrant acidity and expressive, ri...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,NaN,380170
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,2015,90,$45,Chardonnay Dundee Hills,Knudsen,"Crisp and harmonious, with lemon blossom and a...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,NaN,434673
1234,2015,90,$40,Chardonnay Willamette Valley Reserve,Ponzi,"Graceful and polished, with expressive pear, s...","Magazine Issue Jul 31, 2018",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator,Chardonnay,NaN,454785
1235,2015,90,$45,Chardonnay Yamhill-Carlton District Gran Morai...,Raptor Ridge,"Light and polished, showing pretty pear, pinea...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,NaN,427274
1236,2015,90,$60,Chardonnay Yamhill-Carlton District Marsh Esta...,Roco,"Floral and harmonious, with delicate tropical ...",Magazine Issue Web Only - 2018,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,NaN,447448


In [72]:
df.loc[:10,["bottle_name", "winery"]]

,bottle_name,winery
0,Cabernet Franc Rogue Valley,Griffin Creek
1,Arsenal Rogue Valley,Quady North
2,Cabernet Sauvignon Rogue Valley,Del Rio
3,Cabernet Sauvignon Rogue Valley,Griffin Creek
4,Chardonnay Dundee Hills Récolte Grand Cru,Domaine Serene
5,Chardonnay Eola-Amity Hills Seven Springs Vine...,Evening Land
6,Chardonnay Willamette Valley Sigrid,Bergström
7,Chardonnay Dundee Hills Édition Limitée,Domaine Drouhin
8,Chardonnay Dundee Hills Evenstad Reserve,Domaine Serene
9,Chardonnay Dundee Hills,Ayoub


In [73]:
df["varietal"].value_counts()

Chardonnay                     231
Pinot Noir                     190
Pinot Gris                     114
Rosé Wines                      83
Other White Wines               45
Riesling                        37
Shiraz/Syrah                    27
Sparkling Wines                 24
Pinot Blanc                     22
Grenache                        21
Cabernet Sauvignon & Blends     12
Viognier                        10
Sauvignon Blanc                 10
Tempranillo                      9
Other Red Wines                  8
Dessert Wines                    7
Cabernet Franc                   6
Malbec                           3
Grüner Veltliner                 2
Gewürztraminer                   2
Merlot                           2
Name: varietal, dtype: int64

In [76]:
df['winery'].value_counts()

Domaine Serene       57
Bergström            56
Evening Land         45
Beaux Frères         41
Alexana              38
                     ..
Coelho                1
Ancient Cellars       1
Anam Cara Cellars     1
Et Fille              1
Authentique           1
Name: winery, Length: 196, dtype: int64